In [1]:
import os, json
from dotenv import load_dotenv, find_dotenv; _ = load_dotenv(find_dotenv())

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [3]:
topic_type = 'foods'
topic_file = f'{topic_type}.txt'

In [4]:
with open(f'config/{topic_file}', 'r') as f:
    topic_list = f.read().splitlines()
topic_list

['じゃがいも', '玉ねぎ', 'にんじん', 'なす', 'トマト', 'ピーマン']

In [5]:
with open(f'config/config.json', 'r') as f:
    config = json.load(f)
config

{'languages': {'Japanese': 'ja',
  'English': 'en',
  'Chinese': 'zh',
  'German': 'de',
  'Arabic': 'ar',
  'Russian': 'ru'},
 'format': {'prose': '',
  'bullet_list': '情報を構造化して箇条書きに変換してください。',
  'markdown': '階層構造を持つMarkdownに変換してください。各セクション内の文章は散文にしてください。',
  'json': '情報を構造化してJson形式に変換してください。Key-Valueは入力と同じ言語にしてください。',
  'convesation': 'AssistantからUserへ情報を伝える会話形式にしてください。'},
 'length': ['full', 'short_sum', 'long_sum']}

## Transform chain

In [6]:
template_str = """\
ユーザーが入力したテキストを指示された形式に変換してください。
変換後テキストは元のテキストの情報を全て保持するようにしてください。
"""

In [7]:
prompt_template = ChatPromptTemplate.from_messages([
    ('system', template_str),
    ('user', '{text}'),
    ('user', '形式: {format}\n{format_inst}')
])

chat = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0., max_tokens=4096)

transform_chain = prompt_template | chat

### Test

In [8]:
topic = topic_list[0]

res = transform_chain.batch([
    {
        'text': open(f'./data/{topic_type}/{topic}/ja-prose-full.txt', 'r').read(),
        'format': form,
        'format_inst': inst,
    }
    for form, inst in config['format'].items() if form != 'Prose'
])

In [9]:
for r,(t,f) in zip(res, [(typ,form) for typ in config['length'] for form in config['format'] if form != 'Prose']):
    print(t,f,'='*30)
    print(r.content)
    print()

full prose ==============================
じゃがいもは、南アメリカ原産の野菜で、世界中で広く栽培されている主食として知られています。日本でも古くから栽培され、多くの家庭で親しまれている食材です。

じゃがいもは、寒冷な気候での栽培が適しており、日本国内では北海道や青森などの地域で生産されています。栄養価も高く、ビタミンCやカリウム、食物繊維が豊富に含まれています。特に皮ごと食べることで、食物繊維を摂取することができます。

調理方法としては、茹でる、焼く、揚げる、蒸すなどさまざまな方法があります。特にポテトサラダやポテトフライ、ポテトグラタンなど、洋食の定番料理に使われることが多いです。また、和食でも煮物や炒め物に加えられることもあります。

風味は、甘みとほくほくした食感が特徴で、調理方法によってさまざまな味わいを楽しむことができます。旬の季節は秋から冬で、収穫後は日持ちが良いため、長期間保存することができます。

じゃがいもは、肉類や野菜との相性が良く、玉ねぎやニンニク、ローズマリー、オリーブオイルなどとの組み合わせもおすすめです。また、じゃがいもを蒸かしてから調理すると、さらにふんわりとした食感を楽しむことができます。

その他の特徴としては、じゃがいもは皮ごと食べられるため、栄養価を最大限に活かすことができます。また、じゃがいもの種類には、サラダ用や煮崩れしにくい種類などさまざまな種類があり、料理に合わせて選ぶことができます。

full bullet_list ==============================
- じゃがいもは、南アメリカ原産の野菜で、世界中で広く栽培されている主食として知られています。
- 日本でも古くから栽培され、多くの家庭で親しまれている食材です。
- 寒冷な気候での栽培が適しており、日本国内では北海道や青森などの地域で生産されています。
- 栄養価も高く、ビタミンCやカリウム、食物繊維が豊富に含まれています。
- 皮ごと食べることで、食物繊維を摂取することができます。
- 調理方法としては、茹でる、焼く、揚げる、蒸すなどさまざまな方法があります。
- 洋食の定番料理に使われることが多いポテトサラダやポテトフライ、ポテトグラタンなどがあります。
- 和食でも煮物や炒め物に加えられる

### Batch

In [10]:
variables = [
    {
        'topic': topic,
        'length': length,
        'format': form,
        'format_inst': inst,
    }
    for topic in topic_list
    for length in config['length']
    for form, inst in config['format'].items() if form != 'prose'
]

In [11]:
inputs = [
    {
        'text': open(f'./data/{topic_type}/{v["topic"]}/ja-prose-{v["length"]}.txt', 'r').read(),
        'format': v["format"],
        'format_inst': v["format_inst"],
    }
    for v in variables
]
print(len(inputs))

72


In [17]:
# for v in variables:
#     print(v)

In [18]:
res = transform_chain.batch(inputs)

In [14]:
for r, v in zip(res, variables):
    with open(f'./data/{topic_type}/{v["topic"]}/ja-{v["format"]}-{v["length"]}.txt', 'w') as f:
        f.write(r.content)